In [37]:
import os
from pyrsgis import raster
from sklearn import cluster
from osgeo import gdal
import geopandas as gdp
import pandas as pd
import numpy as np
import rasterio as rio
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import matplotlib.pyplot as plt
from glob import glob
from rasterio.plot import show
import pandas as pd


In [38]:
southerAus = 'LC08_L1TP_090083_20191215_20191226_01_T1/LC08_20191215_1226_LayersStacked_ALL.tif'
southerAusBinary = 'LC08_L1TP_090083_20191215_20191226_01_T1/KMeans_LC08_20191215_1226_2clusters_5.tif'
victoriaAus = 'LC08_L1TP_091086_20191222_20200110_01_T1/LC08_20191222_LayersStacked.tiff'

# Read the rasters as array

ds7, featuresSouthAus = raster.read(southerAus, bands='all')
ds6, labelsSouthernAus = raster.read(southerAusBinary, bands=1)
ds5, victoria = raster.read(victoriaAus, bands='all')

In [39]:
print("Fire b7 image shape: ", featuresSouthAus.shape)
print("Fire b6 image shape: ", labelsSouthernAus.shape)
print("Fire b5 image shape: ", victoria.shape)

Fire b7 image shape:  (7, 7691, 7621)
Fire b6 image shape:  (7691, 7621)
Fire b5 image shape:  (7, 7861, 7821)


In [40]:
from pyrsgis.convert import changeDimension

featuresSouthAus = changeDimension(featuresSouthAus)
labelsSouthernAus  = changeDimension(labelsSouthernAus)
victoria = changeDimension(victoria)
nBands = featuresSouthAus.shape[1]
labelsSouthernAuss= (labelsSouthernAus == 1).astype(int)

print("Souther Aus multispectral image shape: ", featuresSouthAus.shape)
print("Southern Aus Fire/nonFire binary shape: ", labelsSouthernAus.shape)
print("Victoria multispectral image shape: ", victoria.shape)

Souther Aus multispectral image shape:  (58613111, 7)
Southern Aus Fire/nonFire binary shape:  (58613111,)
Victoria multispectral image shape:  (61480881, 7)


In [41]:
from sklearn.model_selection import train_test_split

                              #X (Indepent Varibles/Data)  #y(Dependent/Labels (Fire/NonFire))
xTrain, xTest, yTrain, yTest = train_test_split(featuresSouthAus, labelsSouthernAuss, test_size=0.4, random_state=42)

print(xTrain.shape)
print(yTrain.shape)

print(xTest.shape)
print(yTest.shape)

(35167866, 7)
(35167866,)
(23445245, 7)
(23445245,)


In [42]:
# Reshape the data
xTrain = xTrain.reshape((xTrain.shape[0], 1, xTrain.shape[1]))
xTest = xTest.reshape((xTest.shape[0], 1, xTest.shape[1]))
victoria = victoria.reshape((victoria.shape[0], 1, victoria.shape[1]))

# Print the shape of reshaped data
print(xTrain.shape, xTest.shape, victoria.shape)



(35167866, 1, 7) (23445245, 1, 7) (61480881, 1, 7)


In [43]:
from tensorflow import keras

# Define the parameters of the model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1, nBands)),
    keras.layers.Dense(14, activation='relu'),
    keras.layers.Dense(2, activation='softmax')])

# Define the accuracy metrics and parameters
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Run the model
model.fit(xTrain, yTrain, epochs=2)

Train on 35167866 samples
Epoch 1/2
35167866/35167866 [==============================] - 919s 26us/sample - loss: 0.6221 - accuracy: 0.9722
Epoch 2/2
35167866/35167866 [==============================] - 909s 26us/sample - loss: 0.0507 - accuracy: 0.9819


In [44]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

# Predict for test data 
yTestPredicted = model.predict(xTest)
yTestPredicted = yTestPredicted[:,1]

# Calculate and display the error metrics
yTestPredicted = (yTestPredicted>0.5).astype(int)
cMatrix = confusion_matrix(yTest, yTestPredicted)
pScore = precision_score(yTest, yTestPredicted)
rScore = recall_score(yTest, yTestPredicted)

print("Confusion matrix: for 14 nodes\n", cMatrix)
print("\nP-Score: %.3f, R-Score: %.3f" % (pScore, rScore))

Confusion matrix: for 14 nodes
 [[12826292   534477]
 [     856 10083620]]

P-Score: 0.950, R-Score: 1.000


In [53]:
predicted = model.predict(victoria)
predicted = predicted[:,1]
#Export raster
prediction = np.reshape(predicted, (victoria.RasterYSize, victoria.RasterXSize))
outFile = 'Victoria Fire predicted.tif'
raster.export(prediction, ds5, filename=outFile, dtype='float')

UnboundLocalError: local variable 'col' referenced before assignment

In [130]:
# Extracting Values of ROI that have both fire and nonfire
#Reading Training Regions of Interest Pixels
rios = open("RIOs_fire_nonFire_text.txt","r") 
for line in rios:
    print(repr(line))

'; ENVI Output of ROIs (5.3.1) [Fri Apr 10 10:41:01 2020]\n'
'; Number of ROIs: 2\n'
'; File Dimension: 7621 x 7691\n'
';\n'
'; ROI name: Fire Pixels\n'
'; ROI rgb value: {255, 0, 0}\n'
'; ROI npts: 560\n'
';\n'
'; ROI name: Non Fire\n'
'; ROI rgb value: {0, 255, 0}\n'
'; ROI npts: 1357\n'
';   ID     X     Y      Map X       Map Y         Lat         Lon      B1      B2      B3      B4      B5      B6      B7\n'
'     1  3391  4296  231585.00  6311965.00  -33.298020  150.117420  0.1858  0.1762  0.1522  0.1358  0.1882  0.3091  0.8701\n'
'     2  3390  4296  231555.00  6311965.00  -33.298013  150.117098  0.1964  0.1909  0.1672  0.1576  0.2076  0.4163  1.2744\n'
'     3  3392  4296  231615.00  6311965.00  -33.298028  150.117742  0.1900  0.1810  0.1587  0.1414  0.2001  0.2991  0.8800\n'
'     4  3393  4296  231645.00  6311965.00  -33.298035  150.118064  0.2017  0.1931  0.1719  0.1536  0.2136  0.2886  0.9689\n'
'     5  3394  4296  231675.00  6311965.00  -33.298043  150.118385  0.2126  0.2

In [158]:
ROIs = open("RIOs_fire_nonFire_text.txt","r") 

# Read and ignore header lines
header1 = ROIs.readline()
header2 = ROIs.readline()
header3 = ROIs.readline()
header4 = ROIs.readline()
header5 = ROIs.readline()
header6 = ROIs.readline()
header7 = ROIs.readline()
header8 = ROIs.readline()
header9 = ROIs.readline()
header10 = ROIs.readline()
header11 = ROIs.readline()
header12 = ROIs.readline()


values = []

# Loop over lines and extract variables of interest
for line in ROIs:
    line = line.strip()
    #print(line) 
    columns = line.split()
    #print(columns)
    feature_Bands = columns[11:]
   # print(bands)
    print(feature_Bands)
    values.append(feature_Bands)

# Import 2 columns into a dataframe
# Create criteria of table to say first 561 rows are fire and the rest are non fire
  

['0.1882', '0.3091', '0.8701']
['0.2076', '0.4163', '1.2744']
['0.2001', '0.2991', '0.8800']
['0.2136', '0.2886', '0.9689']
['0.2239', '0.4566', '1.3023']
['0.2371', '0.7375', '1.3916']
['0.2428', '0.8290', '1.3916']
['0.2347', '0.7891', '1.3916']
['0.2273', '0.7038', '1.3916']
['0.2174', '0.5882', '1.3916']
['0.1837', '0.3436', '1.3761']
['0.1824', '0.3358', '1.1314']
['0.1802', '0.3098', '1.0997']
['0.1903', '0.3728', '1.3511']
['0.2219', '0.4077', '1.3916']
['0.2131', '0.5787', '1.3916']
['0.2036', '0.5654', '1.3916']
['0.2178', '0.5210', '1.3916']
['0.2203', '0.4154', '1.3916']
['0.2143', '0.3956', '1.3916']
['0.2098', '0.3781', '1.3134']
['0.1976', '0.3183', '1.0002']
['0.2015', '0.4534', '1.2431']
['0.2138', '0.3297', '0.8369']
['0.2254', '0.2688', '0.7335']
['0.2336', '0.3991', '1.3326']
['0.2442', '0.6702', '1.3916']
['0.2433', '0.8735', '1.3916']
['0.2321', '0.6542', '1.3916']
['0.2232', '0.5198', '1.3916']
['0.1805', '0.3705', '1.3865']
['0.1860', '0.3383', '1.2331']
['0.1832